# Qwen3 Tuning
for ARC AGI 2

Updates 29 July 2025:
- Now using utils from the repo.
- Updated lora r to 128, down from 256 as prob 128 is sufficient.

Updates 24 July 2025:
- Now supports calculation of metrics BUT requires validation data have code. This will be updated later to just pass grids for validation.

Updates 23 July 2025:
- Back to constant scheduler for one epoch. Note that SOAR use 3 epochs with cosine scheduler.

Updates 22 July 2025:
- Increased lora r to 256
- Added training on completions only
- Moved to cosine rather than constant scheduler. Moved from 1 to 2 epochs. Note that SOAR use 3 epochs with cosine scheduler.

In [1]:
# !git pull

In [2]:
# !git config --global user.name “RonanMcGovern”
# !git config --global user.email "78278410+RonanKMcGovern@users.noreply.github.com"

In [3]:
import os
os.environ["HF_HOME"] = "/workspace"
os.environ["HF_HUB_CACHE"] = "/workspace/hub" # (recommended) override just the repo cache
print(os.environ["HF_HOME"])

/workspace


### Installation

In [4]:
#To run with vllm.
!uv pip install vllm --system -qU

In [5]:
# Temporary Fix while unsloth is broken! - SHOULD BE FIXED AS OF JULY 20TH 2025
!rm -rf /tmp/unsloth_compiled_cache
!uv pip uninstall trl unsloth --system -q
!uv pip install unsloth -qU --system
!uv pip install trl==0.19.1 --system -q

# sometimes unsloth throws numpy issues
!uv pip uninstall numpy --system -q
!uv pip install numpy==2.2 --system -q
!uv pip show numpy -q

In [6]:
# INSTALLED IN THE CONTAINER IF USING the [arc-agi-2025 container on runpod](https://console.runpod.io/deploy?template=bh0rvngapk&ref=jmfkcdio)
# %%capture
# import os
# !pip install uv -qU
# !uv pip install unsloth matplotlib tensorboard -qU --system
# !export HF_HUB_ENABLE_HF_TRANSFER=1

In [7]:
# # # if you face model download issues
# import os
# os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "0"

In [8]:
from huggingface_hub import HfFolder, login

# Call this at the top of your script / notebook
if HfFolder.get_token() is None:   # no token cached or in $HF_TOKEN
    login()                        # interactive prompt

In [9]:
%load_ext tensorboard
%tensorboard --logdir ./logs --port 6006 --bind_all

### Unsloth

In [10]:
# !uv pip show trl unsloth vllm transformers

In [11]:
!nvidia-smi

Fri Aug  1 14:12:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.03             Driver Version: 550.144.03     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H200                    On  |   00000000:BA:00.0 Off |                    0 |
| N/A   29C    P0             75W /  700W |       1MiB / 143771MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [12]:
import os
import unsloth
from unsloth import FastLanguageModel
import torch

model_slug = "Qwen/Qwen3-4B"
# model_slug = "julien31/Soar-qwen-7b"
# model_slug = "Qwen/Qwen2.5-Coder-7B-Instruct"
# model_slug = "Qwen/Qwen3-30B-A3B"

model_max_length = 32768 #default is ~2k for unsloth!!!
lora_rank = 128

# Training AND validation batch size (incl. for autoregressive train/test example metrics calculations)
batch_size_global = 4 # use 2 for 7/8B, use 4 for 4B on H200.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_slug,
    max_seq_length = model_max_length,   # Context length - can be longer, but uses more memory
    load_in_4bit = False,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
    # cache_dir = '/workspace',
    # token = "hf_...",      # use one if using gated models
    
    # for using fast_inference
    fast_inference = True, # allows for vLLM generation during evaluation
    max_lora_rank=lora_rank,
    gpu_memory_utilization=0.3,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 08-01 14:13:03 [__init__.py:235] Automatically detected platform cuda.
Unsloth: We'll be using `/tmp/unsloth_compiled_cache` for temporary Unsloth patches.
Standard import failed for UnslothDDPOTrainer: No module named 'UnslothDDPOTrainer'. Using tempfile instead!
Unsloth: Patching vLLM v1 graph capture
Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.7.11: Fast Qwen3 patching. Transformers: 4.54.1. vLLM: 0.10.0.
   \\   /|    NVIDIA H200. Num GPUs = 1. Max memory: 139.719 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 9.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Qwen3-4B

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 08-01 14:13:22 [default_loader.py:262] Loading weights took 9.26 seconds
INFO 08-01 14:13:22 [punica_selector.py:19] Using PunicaWrapperGPU.
INFO 08-01 14:13:23 [gpu_model_runner.py:1892] Model loading took 8.0059 GiB and 9.603944 seconds
INFO 08-01 14:13:34 [backends.py:530] Using cache directory: /root/.cache/vllm/torch_compile_cache/4cd45fbc22/rank_0_0/backbone for vLLM's torch.compile
INFO 08-01 14:13:34 [backends.py:541] Dynamo bytecode transform time: 11.02 s
INFO 08-01 14:13:45 [backends.py:161] Directly load the compiled graph(s) for dynamic shape from the cache, took 9.675 s
INFO 08-01 14:14:10 [monitor.py:34] torch.compile takes 11.02 s in total
INFO 08-01 14:14:11 [gpu_worker.py:255] Available KV cache memory: 30.71 GiB
INFO 08-01 14:14:11 [kv_cache_utils.py:833] GPU KV cache size: 223,616 tokens
INFO 08-01 14:14:11 [kv_cache_utils.py:837] Maximum concurrency for 32,768 tokens per request: 6.82x
INFO 08-01 14:14:11 [vllm_utils.py:641] Unsloth: Running patched vLLM v1 `c

Capturing CUDA graph shapes: 100%|██████████| 67/67 [00:20<00:00,  3.29it/s]

INFO 08-01 14:14:31 [gpu_model_runner.py:2485] Graph capturing finished in 20 secs, took 1.00 GiB
INFO 08-01 14:14:31 [vllm_utils.py:648] Unsloth: Patched vLLM v1 graph capture finished in 20 secs.


INFO 08-01 14:14:32 [core.py:193] init engine (profile, create kv cache, warmup model) took 69.47 seconds
Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'post_feedforward_layernorm']
Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'post_feedforward_layernorm']


In [13]:
!nvidia-smi

Fri Aug  1 14:14:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.03             Driver Version: 550.144.03     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H200                    On  |   00000000:BA:00.0 Off |                    0 |
| N/A   27C    P0            113W /  700W |   41950MiB / 143771MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
print(model.max_seq_length)

32768


In [15]:
# # Print a summary of the transformer layers and key dimensions
# for i, block in enumerate(model.model.layers):
#     attn = block.self_attn
#     mlp = block.mlp

#     print(f"Layer {i}:")
#     print(f"  Attention:")
#     print(f"    q_proj: {attn.q_proj.weight.shape}")
#     print(f"    k_proj: {attn.k_proj.weight.shape}")
#     print(f"    v_proj: {attn.v_proj.weight.shape}")
#     print(f"    out_proj: {attn.o_proj.weight.shape}")
#     print(f"  MLP:")
#     print(f"    fc1: {mlp.gate_proj.weight.shape}")
#     print(f"    fc2: {mlp.up_proj.weight.shape}")
#     print(f"    fc3: {mlp.down_proj.weight.shape}")
#     print()

In [16]:
# print(model)

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [17]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128. could consider 128.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                     ],
    lora_alpha = 64,  # Best to choose alpha = rank or rank*2. EXCEPT if using rslora, in which case set it as sqrt(max matrix dimension). 64 is good for Qwen 4B
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

Unsloth 2025.7.11 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [18]:
print(tokenizer.padding_side)

right


In [19]:
# Import utils using standard project root detection
from pathlib import Path
import sys

# Find project root by looking for pyproject.toml
project_root = next(
    (parent for parent in [Path.cwd()] + list(Path.cwd().parents) 
     if (parent / "pyproject.toml").exists()), 
    Path.cwd()
)

# Add llm_python directory to path (where utils is located)
llm_python_dir = project_root / "llm_python"
sys.path.insert(0, str(llm_python_dir))

print(f"📁 Project root: {project_root}")
print(f"📁 Looking for utils in: {llm_python_dir}")

from utils.task_loader import TaskLoader
from utils.scoring import GridScorer, ProgramExecutor
from utils.prompt_utils import create_arc_prompt, extract_python_code
from utils.metrics_utils import calculate_task_metrics, format_metrics_display, metrics_to_percentages
from utils.timeout_utils import execute_with_timeout
from utils.transduction import is_transduction_cheating
from utils.prompt_loader import PromptLoader

# Initialize utility instances
prompt_loader = PromptLoader()
scorer = GridScorer()
print("✅ Utils imported and initialized successfully")

📁 Project root: /workspace/arc-agi-2025
📁 Looking for utils in: /workspace/arc-agi-2025/llm_python
✅ Utils imported and initialized successfully


<a name="Data"></a>
### Data Prep

In [20]:
from pathlib import Path
import json
from typing import Optional
from datasets import load_dataset, DatasetDict

# ---------------------------------------------------------------------
# Config (examples)
# ---------------------------------------------------------------------

# Set max_rows flag to limit train size. None for all
max_rows = None # None for all rows
max_validation_rows = 32

# # CASE 1: single slug with both splits
# train_slug = "Trelis/synth_arc-agi-1_all_training_20250724_131808"
# val_slug = None

# CASE 2: two different slugs
train_slug = "Trelis/soar-20250729_114431-1600"
val_slug   = "Trelis/grids_only_arc-agi-1_all_evaluation_20250729_102313"

enable_thinking = False  # See note in original code

# ---------------------------------------------------------------------
# Prompt management using utils (replacing hard-coded prompts)
# ---------------------------------------------------------------------

# Use prompt_loader to get SOAR prompts from utils
SYSTEM_PROMPT = prompt_loader.get_system_message("soar")
INITIAL_TURN_PROMPT = prompt_loader.get_initial_turn_prompt("soar")

print(f"✅ Using SOAR prompts from utils:")
print(f"   System prompt: {len(SYSTEM_PROMPT)} chars")
print(f"   Initial turn prompt: {len(INITIAL_TURN_PROMPT)} chars")

# Custom formatting functions removed - using create_arc_prompt from utils instead

def hf_dataset_to_chat_dataset(dataset_slug: str, split: str = "train"):
    """
    Convert a HF split into the chat/prompt format using utils.
    """
    ds = load_dataset(dataset_slug, split=split, keep_in_memory=True)

    def select_output(example, output_key, predicted_key):
        """
        Select between regular output and predicted output based on availability and non-zero values.
        """
        # Check if predicted output exists and is non-zero
        if predicted_key in example and example[predicted_key]:
            predicted = example[predicted_key]
            # Check if any predicted output is non-zero (has any non-zero cells)
            if any(any(any(cell != 0 for cell in row) for row in grid) for grid in predicted):
                return predicted
        
        # Fall back to regular output
        return example[output_key]

    def create_chat_messages(example):
        # Select appropriate outputs (predicted if available and non-zero, otherwise regular)
        train_outputs = select_output(example, "train_output", "predicted_train_output")
        test_outputs = select_output(example, "test_output", "predicted_test_output")
        
        # Create task_data in the format expected by create_arc_prompt
        task_data = {
            'train': [{'input': inp, 'output': out} 
                     for inp, out in zip(example["train_input"], train_outputs)],
            'test': [{'input': inp, 'output': out} 
                     for inp, out in zip(example["test_input"], test_outputs)]  # All test examples!
        }
        
        # Use create_arc_prompt from utils (matches run_arc_tasks_soar.py)
        system_content, user_content = create_arc_prompt(task_data, prompt_loader, "soar")

        assistant_content = ""
        if enable_thinking and example.get("reasoning", "").strip():
            assistant_content += f"<think>{example['reasoning'].strip()}</think>"
        assistant_content += f"```python\n{example['code']}\n```"

        messages = [
            {"role": "system", "content": system_content},
            {"role": "user",   "content": user_content},
            {"role": "assistant", "content": assistant_content},
        ]

        prompt_messages = [
            {"role": "system", "content": system_content},
            {"role": "user",   "content": user_content},
        ]

        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False,
            enable_thinking = enable_thinking
        )

        prompt_text = tokenizer.apply_chat_template(
            prompt_messages,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking = enable_thinking
        )

        return {
            "messages": messages,
            "text": text,
            "prompt": prompt_text,
            "train_input": example["train_input"],
            "train_output": train_outputs,  # Use selected outputs (predicted when available)
            "test_input": example["test_input"],
            "test_output": test_outputs,    # Use selected outputs (predicted when available)
            "reasoning": example.get("reasoning", ""),
            "code": example["code"],
            "task_id": example.get("task_id", ""),
        }

    ds = ds.map(create_chat_messages, desc=f"build chat + prompt fields ({split})")
    return ds

def build_dataset(train_slug: str,
                  val_slug: Optional[str] = None,
                  train_split: str = "train",
                  val_split: str = "validation") -> DatasetDict:
    """
    Build a DatasetDict with 'train' and 'validation' keys.
    - If val_slug is None, both splits are loaded from train_slug.
    - Otherwise, load train_split from train_slug and val_split from val_slug.
    """
    
    # Load and filter
    train_ds = hf_dataset_to_chat_dataset(train_slug, split=train_split)
    if max_rows:
        train_ds = train_ds.select(range(min(len(train_ds), max_rows)))
    
    # Validation logic
    if val_slug is None:
        try:
            val_ds = hf_dataset_to_chat_dataset(train_slug, split=val_split)
        except Exception as e:
            raise ValueError(
                f"Could not load split '{val_split}' from '{train_slug}'. "
                f"Pass an explicit val_slug or choose a valid split.\nOriginal error: {e}"
            )
    else:
        val_ds = hf_dataset_to_chat_dataset(val_slug, split="train")
    
    # Corrected slice
    if max_validation_rows:
        val_ds = val_ds.select(range(min(len(val_ds), max_validation_rows)))
    elif max_rows:
        val_ds = val_ds.select(range(min(len(val_ds), max_rows)))
    
    return DatasetDict(train=train_ds, validation=val_ds)

# ---------------------------------------------------------------------
# Build the dataset
# ---------------------------------------------------------------------
data = build_dataset(train_slug, val_slug)  # val_slug may be None 

✅ Using SOAR prompts from utils:
   System prompt: 129 chars
   Initial turn prompt: 990 chars


build chat + prompt fields (train):   0%|          | 0/1600 [00:00<?, ? examples/s]

build chat + prompt fields (train):   0%|          | 0/400 [00:00<?, ? examples/s]

In [21]:
# Validation split
val_ids = [ex["task_id"] for ex in data["validation"]]
assert all(val_ids), "❌ some validation rows are missing task_id"
assert len(val_ids) == len(set(val_ids)), "❌ duplicate task_id in validation slice"

In [22]:
# # print(data["train"][0])
# print(data["train"][0]['prompt'])

In [23]:
# print(data["train"][0]['text'])

In [24]:
print(data["validation"][0]['text'])

<|im_start|>system
You are an AI assistant specialized in solving Abstract Reasoning Corpus (ARC-AGI) tasks by reasoning and generating Python code.<|im_end|>
<|im_start|>user
You are an AI assistant specialized in solving Abstract Reasoning Corpus (ARC-AGI) tasks by generating Python code.
Your goal is to analyze input-output grid pairs. The outputs were produced by applying a transformation rule to the inputs. Implement the transformation rules as a Python function.
You should only write the implemented the transformation in code.
You must write code in triple backticks (```python and then ```). You must write a function called 'transform' which takes a single argument, the input grid as 'list[list[int]]', and returns the transformed grid (also as 'list[list[int]]').
You should make sure that you implement a version of the transformation which works in general (at least for all given input-output pairs and test input pairs).
The number in the input grid can be mapped to the following

Let's see the structure of both datasets:

In [25]:
import numpy as np
from statistics import median

def length_stats(dataset, name=""):
    """
    Return min / median / max tokenised length for a 🤗 Dataset split that has a
    single 'text' column. Uses the same tokenizer already in memory.
    """
    # Tokenise in batches → list of list[int] → list[int] lengths
    lengths = dataset.map(
        lambda batch: {
            "len": [len(ids) for ids in tokenizer(batch["text"],
                                                  add_special_tokens=False
                                                 )["input_ids"]]
        },
        batched=True,
        remove_columns=dataset.column_names,   # drop 'text'
        keep_in_memory=True,
    )["len"]

    print(f"{name:>11}:  min={min(lengths):>4}  "
          f"median={int(median(lengths)):>4}  max={max(lengths):>4}")

# ── run for both splits ────────────────────────────────────────────────────────
length_stats(data["train"],       "train")
length_stats(data["validation"],  "validation")


Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

      train:  min= 518  median=2303  max=19056


Map:   0%|          | 0/32 [00:00<?, ? examples/s]

 validation:  min= 631  median=3561  max=15051


### Pre-Training Data Integrity Tests
Before training, let's test the ground-truth code on a random sample of training examples to validate dataset quality and establish baseline performance.


In [26]:
import random

# Configuration for pre-training tests
NUM_TEST_EXAMPLES = 100  # Number of random examples to test
RANDOM_SEED = 42  # For reproducible results

def run_pre_training_data_integrity_tests(dataset_split="train", num_examples=NUM_TEST_EXAMPLES):
    """
    Test ground-truth code from dataset on random examples to validate data quality.
    
    Args:
        dataset_split: Which split to test (should be "train" since validation has no ground-truth code)
        num_examples: Number of random examples to test
    """
    print(f"🧪 Running Pre-Training Data Integrity Tests")
    print(f"📊 Testing {num_examples} random examples from {dataset_split} split")
    print("=" * 60)
    
    # Set seed for reproducible sampling
    random.seed(RANDOM_SEED)
    
    # Get the dataset split
    dataset = data[dataset_split]
    
    # Randomly sample examples
    total_examples = len(dataset)
    if num_examples > total_examples:
        print(f"⚠️  Requested {num_examples} examples but only {total_examples} available. Testing all.")
        sample_indices = list(range(total_examples))
    else:
        sample_indices = random.sample(range(total_examples), num_examples)
    
    # Initialize tracking variables
    results = []
    executor = ProgramExecutor(timeout=2.0, executor_type="unrestricted")
    
    print(f"\n🔍 Testing {len(sample_indices)} examples...\n")
    
    for i, idx in enumerate(sample_indices):
        example = dataset[idx]
        task_id = example.get("task_id", f"idx_{idx}")
        code = example["code"]
        
        print(f"[{i+1}/{len(sample_indices)}] Testing {task_id}")
        
        # Initialize results for this example
        example_result = {
            "task_id": task_id,
            "index": idx,
            "code": code,
            "train_results": [],
            "test_results": [],
            "train_success": 0,
            "test_success": 0,
            "code_executed": False,
            "errors": []
        }
        
        # Test on training examples
        train_correct = 0
        for t_idx, (train_in, train_out) in enumerate(zip(example["train_input"], example["train_output"])):
            try:
                predicted_output, error, timed_out = executor.execute_program_with_timeout(code, train_in)
                
                if predicted_output is not None:
                    example_result["code_executed"] = True
                    score_result = scorer.score_grid(predicted_output, train_out)
                    is_correct = score_result["correct"]
                    
                    if is_correct:
                        train_correct += 1
                    
                    example_result["train_results"].append({
                        "index": t_idx,
                        "correct": is_correct,
                        "predicted": predicted_output,
                        "expected": train_out,
                        "timed_out": timed_out
                    })
                else:
                    example_result["train_results"].append({
                        "index": t_idx,
                        "correct": False,
                        "error": error,
                        "timed_out": timed_out
                    })
                    if error:
                        example_result["errors"].append(f"Train {t_idx}: {error}")
                        
            except Exception as e:
                example_result["train_results"].append({
                    "index": t_idx,
                    "correct": False,
                    "error": str(e)
                })
                example_result["errors"].append(f"Train {t_idx}: {str(e)}")
        
        # Test on test examples
        test_correct = 0
        for t_idx, (test_in, test_out) in enumerate(zip(example["test_input"], example["test_output"])):
            try:
                predicted_output, error, timed_out = executor.execute_program_with_timeout(code, test_in)
                
                if predicted_output is not None:
                    example_result["code_executed"] = True
                    score_result = scorer.score_grid(predicted_output, test_out)
                    is_correct = score_result["correct"]
                    
                    if is_correct:
                        test_correct += 1
                    
                    example_result["test_results"].append({
                        "index": t_idx,
                        "correct": is_correct,
                        "predicted": predicted_output,
                        "expected": test_out,
                        "timed_out": timed_out
                    })
                else:
                    example_result["test_results"].append({
                        "index": t_idx,
                        "correct": False,
                        "error": error,
                        "timed_out": timed_out
                    })
                    if error:
                        example_result["errors"].append(f"Test {t_idx}: {error}")
                        
            except Exception as e:
                example_result["test_results"].append({
                    "index": t_idx,
                    "correct": False,
                    "error": str(e)
                })
                example_result["errors"].append(f"Test {t_idx}: {str(e)}")
        
        # Calculate success rates for this example
        example_result["train_success"] = train_correct / len(example["train_input"]) if example["train_input"] else 0
        example_result["test_success"] = test_correct / len(example["test_input"]) if example["test_input"] else 0
        
        # Print summary for this example
        total_train = len(example["train_input"])
        total_test = len(example["test_input"])
        
        print(f"  ✅ Train: {train_correct}/{total_train} ({example_result['train_success']:.1%})")
        print(f"  ✅ Test:  {test_correct}/{total_test} ({example_result['test_success']:.1%})")
        
        if example_result["errors"]:
            print(f"  ❌ Errors: {len(example_result['errors'])}")
        if not example_result["code_executed"]:
            print(f"  ⚠️  Code never executed successfully")
        print()
        
        results.append(example_result)
    
    return results

# Run the tests
data_integrity_results = run_pre_training_data_integrity_tests("train", NUM_TEST_EXAMPLES)

🧪 Running Pre-Training Data Integrity Tests
📊 Testing 100 random examples from train split

🔍 Testing 100 examples...

[1/100] Testing 2204b7a8
  ✅ Train: 3/3 (100.0%)
  ✅ Test:  1/1 (100.0%)

[2/100] Testing b782dc8a
  ✅ Train: 2/2 (100.0%)
  ✅ Test:  1/1 (100.0%)

[3/100] Testing 29ec7d0e
  ✅ Train: 4/4 (100.0%)
  ✅ Test:  1/1 (100.0%)

[4/100] Testing be94b721
  ✅ Train: 4/4 (100.0%)
  ✅ Test:  1/1 (100.0%)

[5/100] Testing c0f76784
  ✅ Train: 3/3 (100.0%)
  ✅ Test:  1/1 (100.0%)

[6/100] Testing 94f9d214
  ✅ Train: 4/4 (100.0%)
  ✅ Test:  1/1 (100.0%)

[7/100] Testing 6fa7a44f
  ✅ Train: 4/4 (100.0%)
  ✅ Test:  1/1 (100.0%)

[8/100] Testing ded97339
  ✅ Train: 3/3 (100.0%)
  ✅ Test:  1/1 (100.0%)

[9/100] Testing b94a9452
  ✅ Train: 3/3 (100.0%)
  ✅ Test:  1/1 (100.0%)

[10/100] Testing a79310a0
  ✅ Train: 3/3 (100.0%)
  ✅ Test:  1/1 (100.0%)

[11/100] Testing 5bd6f4ac
  ✅ Train: 4/4 (100.0%)
  ✅ Test:  1/1 (100.0%)

[12/100] Testing be94b721
  ✅ Train: 4/4 (100.0%)
  ✅ Test:  1/1 

In [27]:
def analyze_data_integrity_results(results):
    """
    Analyze and display comprehensive statistics from the data integrity tests.
    """
    print("=" * 60)
    print("📈 PRE-TRAINING DATA INTEGRITY RESULTS ANALYSIS")
    print("=" * 60)
    
    if not results:
        print("❌ No results to analyze!")
        return
    
    # Overall statistics
    total_examples = len(results)
    examples_with_executable_code = sum(1 for r in results if r["code_executed"])
    examples_with_errors = sum(1 for r in results if r["errors"])
    
    # Training performance statistics
    train_success_rates = [r["train_success"] for r in results]
    perfect_train = sum(1 for rate in train_success_rates if rate == 1.0)
    partial_train = sum(1 for rate in train_success_rates if 0 < rate < 1.0)
    failed_train = sum(1 for rate in train_success_rates if rate == 0.0)
    
    # Test performance statistics  
    test_success_rates = [r["test_success"] for r in results]
    perfect_test = sum(1 for rate in test_success_rates if rate == 1.0)
    partial_test = sum(1 for rate in test_success_rates if 0 < rate < 1.0)
    failed_test = sum(1 for rate in test_success_rates if rate == 0.0)
    
    # Calculate overall metrics
    avg_train_success = sum(train_success_rates) / len(train_success_rates) if train_success_rates else 0
    avg_test_success = sum(test_success_rates) / len(test_success_rates) if test_success_rates else 0
    
    # Count total grids tested
    total_train_grids = sum(len(r["train_results"]) for r in results)
    total_test_grids = sum(len(r["test_results"]) for r in results)
    correct_train_grids = sum(sum(tr["correct"] for tr in r["train_results"]) for r in results)
    correct_test_grids = sum(sum(tr["correct"] for tr in r["test_results"]) for r in results)
    
    print(f"\n🎯 OVERALL PERFORMANCE:")
    print(f"   Examples tested: {total_examples}")
    print(f"   Code executable: {examples_with_executable_code}/{total_examples} ({examples_with_executable_code/total_examples:.1%})")
    print(f"   Examples with errors: {examples_with_errors}/{total_examples} ({examples_with_errors/total_examples:.1%})")
    
    print(f"\n📊 TRAINING GRIDS PERFORMANCE:")
    print(f"   Average success rate: {avg_train_success:.1%}")
    print(f"   Perfect examples (100%): {perfect_train}/{total_examples} ({perfect_train/total_examples:.1%})")
    print(f"   Partial examples (>0% <100%): {partial_train}/{total_examples} ({partial_train/total_examples:.1%})")
    print(f"   Failed examples (0%): {failed_train}/{total_examples} ({failed_train/total_examples:.1%})")
    print(f"   Grid-level accuracy: {correct_train_grids}/{total_train_grids} ({correct_train_grids/total_train_grids:.1%})")
    
    print(f"\n🎯 TEST GRIDS PERFORMANCE:")
    print(f"   Average success rate: {avg_test_success:.1%}")
    print(f"   Perfect examples (100%): {perfect_test}/{total_examples} ({perfect_test/total_examples:.1%})")
    print(f"   Partial examples (>0% <100%): {partial_test}/{total_examples} ({partial_test/total_examples:.1%})")
    print(f"   Failed examples (0%): {failed_test}/{total_examples} ({failed_test/total_examples:.1%})")
    print(f"   Grid-level accuracy: {correct_test_grids}/{total_test_grids} ({correct_test_grids/total_test_grids:.1%})")
    
    # Detailed breakdown by example
    print(f"\n📋 DETAILED BREAKDOWN BY EXAMPLE:")
    print("-" * 60)
    
    for i, result in enumerate(results):
        task_id = result["task_id"]
        train_rate = result["train_success"]
        test_rate = result["test_success"]
        executed = "✅" if result["code_executed"] else "❌"
        error_count = len(result["errors"])
        
        print(f"[{i+1:2d}] {task_id}")
        print(f"     Train: {train_rate:5.1%} | Test: {test_rate:5.1%} | Executed: {executed} | Errors: {error_count}")
        
        if result["errors"] and len(result["errors"]) <= 3:  # Show first few errors
            for error in result["errors"][:3]:
                print(f"     Error: {error}")
        elif len(result["errors"]) > 3:
            print(f"     Errors: {result['errors'][0]} ... (+{len(result['errors'])-1} more)")
    
    # Quality assessment
    print(f"\n🔍 DATASET QUALITY ASSESSMENT:")
    print("-" * 60)
    
    if avg_train_success > 0.9:
        print("✅ EXCELLENT: Ground-truth code performs very well on training examples")
    elif avg_train_success > 0.7:
        print("✅ GOOD: Ground-truth code performs well on training examples")
    elif avg_train_success > 0.5:
        print("⚠️  MODERATE: Ground-truth code has mixed performance on training examples")
    else:
        print("❌ POOR: Ground-truth code has low performance on training examples")
    
    if avg_test_success > 0.9:
        print("✅ EXCELLENT: Ground-truth code generalizes very well to test examples")
    elif avg_test_success > 0.7:
        print("✅ GOOD: Ground-truth code generalizes well to test examples")
    elif avg_test_success > 0.5:
        print("⚠️  MODERATE: Ground-truth code has mixed generalization to test examples")
    else:
        print("❌ POOR: Ground-truth code has poor generalization to test examples")
    
    if examples_with_executable_code == total_examples:
        print("✅ EXCELLENT: All ground-truth code is executable")
    elif examples_with_executable_code / total_examples > 0.9:
        print("✅ GOOD: Most ground-truth code is executable")
    else:
        print("⚠️  ISSUE: Some ground-truth code is not executable")
    
    print("\n" + "=" * 60)
    
    return {
        "total_examples": total_examples,
        "executable_rate": examples_with_executable_code / total_examples,
        "avg_train_success": avg_train_success,
        "avg_test_success": avg_test_success,
        "perfect_train_rate": perfect_train / total_examples,
        "perfect_test_rate": perfect_test / total_examples,
        "train_grid_accuracy": correct_train_grids / total_train_grids if total_train_grids > 0 else 0,
        "test_grid_accuracy": correct_test_grids / total_test_grids if total_test_grids > 0 else 0
    }

# Analyze the results
summary_stats = analyze_data_integrity_results(data_integrity_results)


📈 PRE-TRAINING DATA INTEGRITY RESULTS ANALYSIS

🎯 OVERALL PERFORMANCE:
   Examples tested: 100
   Code executable: 100/100 (100.0%)
   Examples with errors: 0/100 (0.0%)

📊 TRAINING GRIDS PERFORMANCE:
   Average success rate: 100.0%
   Perfect examples (100%): 100/100 (100.0%)
   Partial examples (>0% <100%): 0/100 (0.0%)
   Failed examples (0%): 0/100 (0.0%)
   Grid-level accuracy: 313/313 (100.0%)

🎯 TEST GRIDS PERFORMANCE:
   Average success rate: 100.0%
   Perfect examples (100%): 100/100 (100.0%)
   Partial examples (>0% <100%): 0/100 (0.0%)
   Failed examples (0%): 0/100 (0.0%)
   Grid-level accuracy: 100/100 (100.0%)

📋 DETAILED BREAKDOWN BY EXAMPLE:
------------------------------------------------------------
[ 1] 2204b7a8
     Train: 100.0% | Test: 100.0% | Executed: ✅ | Errors: 0
[ 2] b782dc8a
     Train: 100.0% | Test: 100.0% | Executed: ✅ | Errors: 0
[ 3] 29ec7d0e
     Train: 100.0% | Test: 100.0% | Executed: ✅ | Errors: 0
[ 4] be94b721
     Train: 100.0% | Test: 100.0% | E

In [28]:
# Note: Validation set typically doesn't have ground-truth programs, so we only test training set

# Function to examine specific failing examples in detail
def examine_failure(results, example_index):
    """Examine a specific failing example in detail."""
    if example_index >= len(results):
        print(f"❌ Invalid index {example_index}. Only {len(results)} examples available.")
        return
    
    result = results[example_index]
    print(f"\n🔍 DETAILED EXAMINATION: Example {example_index + 1}")
    print(f"Task ID: {result['task_id']}")
    print("=" * 50)
    
    print(f"\n📝 GROUND TRUTH CODE:")
    print("-" * 30)
    print(result['code'])
    
    print(f"\n📊 EXECUTION SUMMARY:")
    print(f"Code executed successfully: {result['code_executed']}")
    print(f"Train success rate: {result['train_success']:.1%}")
    print(f"Test success rate: {result['test_success']:.1%}")
    print(f"Number of errors: {len(result['errors'])}")
    
    if result['errors']:
        print(f"\n❌ ERRORS:")
        for i, error in enumerate(result['errors']):
            print(f"  {i+1}. {error}")

# Check for failing examples
failed_examples = [i for i, r in enumerate(data_integrity_results) 
                  if r['train_success'] < 1.0 or r['test_success'] < 1.0 or not r['code_executed']]

print(f"\n🔍 FAILING EXAMPLES SUMMARY:")
if failed_examples:
    print(f"Found {len(failed_examples)} examples with issues: {failed_examples}")
    print("To examine a specific failure, run: examine_failure(data_integrity_results, index)")
else:
    print("🎉 No failing examples found! All ground-truth code works perfectly.")

print(f"\n✅ Pre-training data integrity tests complete!")
print(f"📋 Summary stats saved in 'summary_stats' variable")
print(f"📊 Detailed results saved in 'data_integrity_results' variable")


🔍 FAILING EXAMPLES SUMMARY:
🎉 No failing examples found! All ground-truth code works perfectly.

✅ Pre-training data integrity tests complete!
📋 Summary stats saved in 'summary_stats' variable
📊 Detailed results saved in 'data_integrity_results' variable


In [29]:
examine_failure(data_integrity_results, 11)


🔍 DETAILED EXAMINATION: Example 12
Task ID: be94b721

📝 GROUND TRUTH CODE:
------------------------------
import numpy as np

def transform(grid_lst: list[list[int]]) -> list[list[int]]:
    grid = np.array(grid_lst)
    rows, cols = grid.shape
    unique_elements = np.unique(grid)
    if len(unique_elements) <= 1:
        return grid.tolist()
    counts = np.bincount(grid.flatten())
    counts[0] = 0
    most_frequent_element = np.argmax(counts)
    non_zero_coords = np.argwhere(grid == most_frequent_element)
    min_row, min_col = non_zero_coords.min(axis=0)
    max_row, max_col = non_zero_coords.max(axis=0)
    subgrid = grid[min_row:max_row + 1, min_col:max_col + 1]
    return subgrid.tolist()

📊 EXECUTION SUMMARY:
Code executed successfully: True
Train success rate: 100.0%
Test success rate: 100.0%
Number of errors: 0


### Training Setup
Now we'll set up the trainer and then validate all evaluation components.

In [30]:
# print(help(model.fast_generate))

In [31]:
from datetime import datetime
import re

# Extract date and time using regex
print("Extract training set date and time as dataset identifiers")

# Try pattern 1: timestamp at end (original pattern)
match = re.search(r'(\d{8}_\d{6})$', train_slug)
if match:
    timestamp = match.group(1)
    date_str = timestamp[:8]
    time_str = timestamp[9:]
    print(f"Date: {date_str} (YYYYMMDD)")
    print(f"Time: {time_str} (HHMMSS)")
else:
    # Try pattern 2: SOAR dataset format (soar-YYYYMMDD_HHMMSS-rows)
    match = re.search(r'soar-(\d{8}_\d{6})-\d+', train_slug)
    if match:
        timestamp = match.group(1)
        date_str = timestamp[:8]
        time_str = timestamp[9:]
        print(f"Date: {date_str} (YYYYMMDD)")
        print(f"Time: {time_str} (HHMMSS)")
    else:
        print("No timestamp found.")
        date_str = "unknown"
        time_str = "unknown"

run_name = f"{model_slug.split('/')[-1]}-ds{date_str}_{time_str}-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
print(f"Run name will be {run_name}")

Extract training set date and time as dataset identifiers
Date: 20250729 (YYYYMMDD)
Time: 114431 (HHMMSS)
Run name will be Qwen3-4B-ds20250729_114431-20250801-141514


In [32]:
# Legacy code extraction imports - now using utils.prompt_utils

In [33]:
import torch, subprocess, os, gc, time

def _print_gpu(prefix=""):
    alloc = torch.cuda.memory_allocated() / 2**20  # MiB
    reserved = torch.cuda.memory_reserved() / 2**20
    print(f"{prefix}CUDA‑alloc={alloc:.0f} MiB | reserved={reserved:.0f} MiB")

def _nvidia_smi():
    try:
        smi = subprocess.check_output(
            ["nvidia-smi", "--query-gpu=memory.used,memory.free",
             "--format=csv,noheader,nounits"]).decode().strip()
        print("nvidia-smi (used/free MiB):", smi)
    except Exception:
        pass  # nvidia-smi not always available


TEMPLATES = {
    "llama": (
        "<|start_header_id|>user<|end_header_id|>\n\n",
        "<|start_header_id|>assistant<|end_header_id|>\n\n",
    ),
    "gemma": (
        "<start_of_turn>user\n",
        "<start_of_turn>model\n",
    ),
    "qwen-coder": (
        "<|im_start|>user\n",
        "<|im_start|>assistant\n", # this is actually how you properly allow the model to keep reasoning!
    ),
    "qwen": (
        "<|im_start|>user\n",
        "<|im_start|>assistant\n<think>\n\n</think>\n\n", # this is actually how you properly allow the model to keep reasoning!
    ),
    "mistral": (
        "[INST]",
        "[/INST]",
    )
}

# instruction_tag, response_tag = TEMPLATES["qwen-coder"]   # ← change if needed and comment out below

model_slug_lower = model_slug.lower()

if "qwen" in model_slug_lower:
    if "coder" in model_slug_lower:
        instruction_tag, response_tag = TEMPLATES["qwen-coder"]
    elif "soar-qwen" in model_slug_lower:
        instruction_tag, response_tag = TEMPLATES["qwen-coder"]
    else:
        instruction_tag, response_tag = TEMPLATES["qwen"]
else:
    raise ValueError(f"Unsupported model slug for Qwen template: {model_slug}")

In [34]:
print(f"Response tag selected: {response_tag}")

Response tag selected: <|im_start|>assistant
<think>

</think>




In [35]:
# VllmMemoryCallback removed - not compatible with vLLM training setup
# Smaller batch size support
import torch
from trl import SFTTrainer
from vllm import SamplingParams

MAX_NEW_TOKENS = 1000  # one place to tune

class VllmSFTTrainer(SFTTrainer):
    def prediction_step(self, model, inputs, prediction_loss_only=False, ignore_keys=None):
        print(f"[vLLM predict] batch size = {inputs['input_ids'].shape[0]}, generating {NUM_EVAL_ATTEMPTS} attempts per task")
        ids = inputs["input_ids"].to(self.args.device)

        # 1) Find prompts up to response_part
        response_tag_ids = self.tokenizer(response_tag, add_special_tokens=False)["input_ids"]
        def _find_sublist(lst, sublst):
            L, M = len(lst), len(sublst)
            for i in range(L - M + 1):
                if lst[i : i + M] == sublst:
                    return i
            raise ValueError

        prompts = []
        for batch_idx, row in enumerate(ids):
            row_ids = row.tolist()
            try:
                start = _find_sublist(row_ids, response_tag_ids)
                cut   = start + len(response_tag_ids)
            except ValueError:
                cut = len(row_ids)
            text = self.tokenizer.decode(
                row_ids[:cut],
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True
            ).strip()
            if not text:
                print(f"[prompt‑dbg] row {batch_idx}: empty → using response_part only")
                text = response_tag
            prompts.append(text)

        # 2) Generate NUM_EVAL_ATTEMPTS responses per prompt with multi-attempt sampling
        # Use same parameters as compute_arc_metrics for consistency
        params = SamplingParams(temperature=1.0, min_p=0.05, max_tokens=MAX_NEW_TOKENS)
        
        all_gen_seqs = []
        for attempt in range(NUM_EVAL_ATTEMPTS):
            with torch.no_grad():
                outs = model.fast_generate(prompts, sampling_params=params)
            
            # Convert each output to a 1D LongTensor
            gen_seqs = [
                torch.tensor(o.outputs[0].token_ids, dtype=torch.long)
                for o in outs
            ]
            all_gen_seqs.extend(gen_seqs)

        # 3) Manually pad or truncate *every* sequence to EXACTLY MAX_NEW_TOKENS
        pad_id = self.tokenizer.pad_token_id
        total_sequences = len(all_gen_seqs)  # batch_size * NUM_EVAL_ATTEMPTS
        
        # allocate [total_sequences, MAX_NEW_TOKENS] filled with pad_id
        fixed = all_gen_seqs[0].new_full((total_sequences, MAX_NEW_TOKENS), pad_id)
        for i, seq in enumerate(all_gen_seqs):
            length = min(seq.size(0), MAX_NEW_TOKENS)
            fixed[i, :length] = seq[:length]

        gen_ids = fixed.cpu()  # shape = [batch_size * NUM_EVAL_ATTEMPTS, MAX_NEW_TOKENS]

        # 4) Return in HF–Trainer's expected format
        # Note: compute_metrics will need to handle the expanded batch size
        return (None, gen_ids, inputs.get("labels"))

In [36]:
DEBUG = False           # ← flip this or pass debug=True below
NUM_EVAL_ATTEMPTS = 8   # Number of attempts per task during evaluation

def compute_arc_metrics(eval_pred, *, debug: bool = DEBUG):
    """
    Multi-attempt evaluation with predictions from VllmSFTTrainer:
    • Uses NUM_EVAL_ATTEMPTS predictions per task from prediction_step
    • Supports any number of test grids per task
    • Uses voting/best-attempt metrics across multiple attempts
    • Tracks comprehensive metrics like run_arc_tasks_soar.py
    """
    print(f"🎯 MULTI-ATTEMPT EVALUATION: {NUM_EVAL_ATTEMPTS} attempts per task from trainer prediction_step")
    
    preds, _ = eval_pred
    
    # preds now contains NUM_EVAL_ATTEMPTS responses per task
    total_predictions = preds.shape[0]
    num_tasks = total_predictions // NUM_EVAL_ATTEMPTS
    raw_dataset = trainer.eval_dataset

    print(f"📊 Processing {total_predictions} predictions for {num_tasks} tasks ({NUM_EVAL_ATTEMPTS} attempts each)")

    # --- Convert "column-dict" → "list of row-dicts" when needed ----
    if isinstance(raw_dataset, dict):
        n = len(next(iter(raw_dataset.values())))
        raw_dataset = [{k: v[i] for k, v in raw_dataset.items()} for i in range(n)]

    scorer = GridScorer()
    executor = ProgramExecutor(timeout=0.5, executor_type="unrestricted")
    
    results = []
    
    for task_idx in range(num_tasks):
        ex = raw_dataset[task_idx]
        
        if debug:
            print(f"\n=== ARC task {task_idx} - processing {NUM_EVAL_ATTEMPTS} pre-generated attempts ===")
        
        # Get all attempts for this task
        start_idx = task_idx * NUM_EVAL_ATTEMPTS
        end_idx = start_idx + NUM_EVAL_ATTEMPTS
        task_predictions = preds[start_idx:end_idx]
        
        # Create task data in the format expected by calculate_task_metrics
        task_data = {
            "task_id": ex.get("task_id", f"task_{task_idx}"),
            "train": [
                {"input": inp, "output": out} 
                for inp, out in zip(ex["train_input"], ex["train_output"])
            ],
            "test": [
                {"input": inp, "output": out} 
                for inp, out in zip(ex["test_input"], ex["test_output"])
            ]
        }
        
        attempt_details = []
        
        for attempt_num in range(NUM_EVAL_ATTEMPTS):
            pred_tokens = task_predictions[attempt_num]
            
            # Decode the prediction
            try:
                # Remove padding tokens
                non_pad_mask = pred_tokens != tokenizer.pad_token_id
                if non_pad_mask.any():
                    clean_tokens = pred_tokens[non_pad_mask]
                    generated_text = tokenizer.decode(clean_tokens, skip_special_tokens=True)
                else:
                    generated_text = ""
                
                # Extract and evaluate code
                code = extract_python_code(generated_text)
                program_extracted = bool(code and code.strip())
                
                # DEBUG: Check what we're actually generating
                if debug or task_idx < 2:  # Always debug first 2 tasks
                    print(f"    Attempt {attempt_num}: Generated text preview: {generated_text[:200]}...")
                    print(f"    Extracted code: {code[:100] if code else 'None'}...")
                    print(f"    Program extracted: {program_extracted}")
                
                # Initialize attempt tracking
                train_results = []
                test_results = []
                test_predictions = []
                any_timeout = False
                code_ran = False
                
                # Evaluate on training examples
                train_correct_count = 0
                for tr_in, tr_out in zip(ex["train_input"], ex["train_output"]):
                
                    if program_extracted:
                        pred, err, timed_out = executor.execute_program_with_timeout(code, tr_in)
                        code_ran |= (err == "")
                    else:
                        pred, err, timed_out = None, "no program", False
                
                    any_timeout |= timed_out
                
                    is_correct = (
                        pred is not None and
                        scorer.score_grid(pred, tr_out)["correct"]   # <-- Boolean!
                    )
                
                    if is_correct:
                        train_correct_count += 1
                
                    train_results.append(
                        {
                            "correct":    is_correct,   # MUST be a bool
                            "prediction": pred,
                            "error":      err,
                            "timeout":    timed_out,
                        }
                    )
                
                # Evaluate on test examples  
                # ---------- TEST loop ----------
                test_correct_count = 0
                test_predictions   = []
                
                for te_in, te_out in zip(ex["test_input"], ex["test_output"]):
                
                    if program_extracted:
                        pred, err, timed_out = executor.execute_program_with_timeout(code, te_in)
                        code_ran |= (err == "")
                    else:
                        pred, err, timed_out = None, "no program", False
                
                    any_timeout |= timed_out
                    test_predictions.append(pred)
                
                    is_correct = (
                        pred is not None and
                        scorer.score_grid(pred, te_out)["correct"]
                    )
                
                    if is_correct:
                        test_correct_count += 1
                
                    test_results.append(
                        {
                            "correct":    is_correct,
                            "prediction": pred,
                            "error":      err,
                            "timeout":    timed_out,
                        }
                    )
                
                # Check for transduction before creating attempt detail
                if program_extracted and code:
                    from utils.transduction import is_transduction_cheating
                    is_cheat, _ = is_transduction_cheating(code, task_data)
                else:
                    is_cheat = False  # No program means not transductive
                
                # Create attempt detail
                attempt_detail = {
                    "task_id": ex.get("task_id", f"task_{task_idx}"),
                    "attempt": attempt_num,
                    "program": code,
                    "program_extracted": program_extracted,
                    "code_ran": code_ran,
                    "train_results": train_results,
                    "test_results": test_results, 
                    "test_predicted": test_predictions[0] if test_predictions else None,  # Single prediction, not list
                    "all_train_correct": train_correct_count == len(ex["train_input"]),
                    "all_test_correct": test_correct_count == len(ex["test_input"]),
                    "any_timeout": any_timeout,
                    "max_length": len(clean_tokens) >= MAX_NEW_TOKENS - 10,  # Near max length
                    "response_text": generated_text[:200] + "..." if len(generated_text) > 200 else generated_text,
                    "train_accuracy":  train_correct_count / len(ex["train_input"]),
                    "is_transductive": is_cheat,                              # <- prevent re-calculation
                }
                
            except Exception as e:
                if debug:
                    print(f"  Task {task_idx}, attempt {attempt_num} failed: {e}")
                
                # Create failed attempt
                attempt_detail = {
                    "task_id": ex.get("task_id", f"task_{task_idx}"),
                    "attempt": attempt_num,
                    "program": "",
                    "program_extracted": False,
                    "code_ran": False,
                    "train_results": [
                        {"correct": False, "prediction": None, "error": str(e), "timeout": False}
                        for _ in ex["train_input"]
                    ],
                    "test_results": [
                        {"correct": False, "prediction": None, "error": str(e), "timeout": False}
                        for _ in ex["test_input"]
                    ],
                    "test_predictions": [None for _ in ex["test_input"]],
                    # ───── add the two fields that voting_utils expects ─────
                    "test_predicted":  [None for _ in ex["test_input"]],  # <- alias
                    "train_accuracy":  0.0,                               # <- weight = 1.0
                    # ────────────────────────────────────────────────────────
                    "all_train_correct": False,
                    "all_test_correct":  False,
                    "any_timeout":       False,
                    "max_length":        False,
                    "response_text":     f"Error: {e}",
                    "is_transductive":   False,                           # <- failed attempts are not transductive
                }
            
            attempt_details.append(attempt_detail)
        
        # Create task result
        task_result = {"task_data": task_data, "attempt_details": attempt_details}
        results.append(task_result)
        
        if debug:
            successful_attempts = sum(1 for att in attempt_details if att["all_test_correct"])
            code_success_attempts = sum(1 for att in attempt_details if att["code_ran"])
            
            # DEBUG: Show train accuracies for each attempt
            train_accs = [att["train_accuracy"] for att in attempt_details]
            train_acc_summary = f"train_accs=[{', '.join(f'{acc:.2f}' for acc in train_accs)}]"
            
            print(f"  Task {task_idx} summary: {successful_attempts}/{NUM_EVAL_ATTEMPTS} test-correct, {code_success_attempts}/{NUM_EVAL_ATTEMPTS} code-ran")
            print(f"  Train accuracies: {train_acc_summary}")

    # -------------- AGGREGATE METRICS (using same logic as run_arc_tasks_soar.py) ------
    print("\n🔍 DEBUG: Checking data structure for voting...")
    for i, result in enumerate(results[:1]):  # Just check first task
        print(f"\nTask {i}:")
        print(f"  task_data keys: {result['task_data'].keys()}")
        print(f"  num attempts: {len(result['attempt_details'])}")
        
        for j, att in enumerate(result['attempt_details'][:2]):  # First 2 attempts
            print(f"  Attempt {j}:")
            print(f"    Keys: {list(att.keys())}")
            print(f"    all_test_correct: {att.get('all_test_correct')}")
            print(f"    all_train_correct: {att.get('all_train_correct')}")
            print(f"    program_extracted: {att.get('program_extracted')}")
            print(f"    code_ran: {att.get('code_ran')}")
            if att.get('test_results'):
                print(f"    test_results sample: {att['test_results'][0]}")
    
    # DEBUG: Check what predictions look like for voting
    print("\n🔍 VOTING DEBUG: Checking correct predictions for voting compatibility...")
    for i, result in enumerate(results[:1]):  # Just check first task  
        correct_attempts = [att for att in result['attempt_details'] if att.get('all_test_correct')]
        print(f"\nTask {i}: {len(correct_attempts)} correct attempts out of {len(result['attempt_details'])}")
        
        if correct_attempts:
            from utils.voting_utils import serialize_prediction_for_voting
            print(f"  Correct predictions for voting:")
            for j, att in enumerate(correct_attempts):
                pred = att.get('test_predicted')
                serialized = serialize_prediction_for_voting(pred)
                print(f"    Attempt {j}: type={type(pred)}, value={pred}")
                print(f"    Serialized key: {serialized[:150]}...")  # First 150 chars
            
            # Check if serialized keys are identical
            serialized_keys = [serialize_prediction_for_voting(att.get('test_predicted')) for att in correct_attempts]
            unique_keys = set(serialized_keys)
            print(f"  Unique serialized keys: {len(unique_keys)} (should be 1 for voting to work)")
            if len(unique_keys) > 1:
                print(f"  ❌ VOTING ISSUE: Different serialized keys for same task!")
                for k, key in enumerate(unique_keys):
                    print(f"    Key {k}: {key[:100]}...")
            
            # DEBUG: Test voting functions directly
            print(f"\n  🗳️  VOTING FUNCTION TESTS:")
            from utils.voting_utils import compute_weighted_majority_voting, compute_train_majority_voting
            from utils.voting_utils import filter_non_transductive_attempts
            
            # Filter non-transductive attempts (as metrics_utils does)
            non_trans = filter_non_transductive_attempts(result)
            print(f"    Non-transductive attempts: {len(non_trans)}/{len(result['attempt_details'])}")
            
            if non_trans:
                # Test weighted voting
                weighted_results = compute_weighted_majority_voting(non_trans, top_k=2)
                print(f"    Weighted voting returned: {len(weighted_results)} predictions")
                if weighted_results:
                    print(f"    Weighted top prediction: {weighted_results[0]}")
                
                # Test train majority voting  
                train_maj_results = compute_train_majority_voting(non_trans, top_k=2)
                print(f"    Train majority voting returned: {len(train_maj_results)} predictions")
                if train_maj_results:
                    print(f"    Train majority top prediction: {train_maj_results[0]}")
                
                # Show ground truth for comparison
                gt = tuple(t["output"] for t in result["task_data"]["test"])
                print(f"    Ground truth: {gt}")
                
                # Show how comparison works
                if weighted_results:
                    from utils.metrics_utils import _to_tuple
                    # Handle both single test (raw grid) and multi-test (tuple of grids) cases
                    for j, pred in enumerate(weighted_results):
                        if pred is None:
                            continue
                        # If it's already a tuple (multi-test), keep as is
                        # If it's a single grid, convert to tuple to match gt format
                        if len(gt) == 1:
                            # Single test case: pred should be raw grid, wrap in tuple
                            normalized_pred = _to_tuple([pred])
                        else:
                            # Multi test case: pred should already be tuple
                            normalized_pred = _to_tuple(pred)
                        
                        matches = (normalized_pred == gt)
                        print(f"    Weighted pred {j}: {normalized_pred} == {gt} ? {matches}")
                
            else:
                print(f"    ❌ No non-transductive attempts to vote on!")
    
    # DEBUG: Check transduction filtering before calling calculate_task_metrics
    print("\n🔍 DEBUG: Checking transduction filtering...")
    for i, result in enumerate(results[:2]):  # Check first 2 tasks
        attempts = result["attempt_details"]
        print(f"\nTask {i}: {len(attempts)} total attempts")
        
        # Check if attempts have is_transductive flag
        has_flag = any('is_transductive' in att for att in attempts)
        print(f"  Has is_transductive flag: {has_flag}")
        
        # Test transduction filtering
        from utils.voting_utils import filter_non_transductive_attempts
        non_trans = filter_non_transductive_attempts(result)
        print(f"  Non-transductive attempts: {len(non_trans)}/{len(attempts)}")
        
        if len(non_trans) == 0 and len(attempts) > 0:
            print("  ⚠️  All attempts filtered out! Testing transduction detection...")
            from utils.transduction import is_transduction_cheating
            for j, att in enumerate(attempts[:3]):  # Check first 3 attempts
                if att.get('program'):
                    is_cheat, reason = is_transduction_cheating(att['program'], result['task_data'], debug=True)
                    print(f"    Attempt {j}: is_transductive={is_cheat}, reason='{reason}'")
                    if is_cheat:
                        print(f"    Program preview: {att['program'][:200]}...")

    metrics = calculate_task_metrics(results, max_tokens=MAX_NEW_TOKENS)
    
    tot = max(1, metrics["total"])
    tresp = max(1, metrics["total_responses"])

    # Return metrics compatible with trainer expectations  
    final_metrics = {
        "all_test_correct": metrics["all_test_correct"] / tot,           # Oracle (best attempt)
        "all_train_correct": metrics["all_train_correct"] / tot,         # Perfect train performance
        "min1_train_correct": metrics["min1_train_correct"] / tot,       # At least some train success
        "min1_code_success": metrics["min1_code_success"] / tot,         # At least one working program
        "weighted_voting_pass2": metrics.get("weighted_pass2", 0) / tot,    # Voting metrics
        "train_majority_pass2": metrics.get("train_majority_pass2", 0) / tot,      # Train majority voting
        "max_length_rate": metrics["max_length_responses"] / tresp,      # Max length hit rate
        "timeout_rate": metrics["timeout_responses"] / tresp,            # Timeout rate
        "total_tasks": tot,
        "total_responses": tresp,
        "total_attempts": tot * NUM_EVAL_ATTEMPTS,
    }
    
    if debug:
        print(f"\n🎯 FINAL MULTI-ATTEMPT METRICS:")
        for k, v in final_metrics.items():
            print(f"  {k}: {v}")
    
    return final_metrics

In [37]:
# Update the trainer configuration to use the fixed custom metrics
from trl import SFTTrainer, SFTConfig

# make sure the attr exists so Unsloth can safely delete it
setattr(model, "_flag_for_generation", True)

trainer = VllmSFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=data["train"],
    eval_dataset=data["validation"],
    args=SFTConfig(
        dataset_text_field="text",
        per_device_train_batch_size = batch_size_global,
        per_device_eval_batch_size = len(data["validation"]),
        # per_device_eval_batch_size = batch_size_global,
        gradient_accumulation_steps=int(32 / batch_size_global),
        warmup_steps=5,
        eval_steps=0.1,
        do_eval=True,
        eval_strategy="steps",
        num_train_epochs=3,
        # max_steps=3,
        learning_rate=1e-4,
        logging_steps=0.05,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="constant",
        seed=3407,
        report_to="tensorboard",
        logging_dir=f"./logs/{run_name}",
        remove_unused_columns=True,
        save_strategy="steps",
        save_steps=0.2,
        save_total_limit=3,
        prediction_loss_only=False
    ),
    compute_metrics=compute_arc_metrics,
    # callbacks=[VllmMemoryCallback()], # best not to mess with vllm
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/1600 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/32 [00:00<?, ? examples/s]

### Evaluation Metrics and Functions Testing
Now that the trainer is set up, let's test all evaluation components to ensure they work correctly with the actual training setup.


In [38]:
from transformers.trainer_utils import EvalPrediction

# Import existing tests from utils/tests instead of duplicating logic
# Note: Assumes we're running from the llm_python directory structure

# Test Configuration
print("🧪 TESTING EVALUATION METRICS AND FUNCTIONS")
print("=" * 60)

def run_existing_tests():
    """Run existing tests from utils/tests to validate core functionality"""
    print("\n📋 Running existing tests from utils/tests...")
    
    test_results = {}
    
    # Test extract_python_code using existing tests
    print("\n🐍 Testing extract_python_code (from utils/tests)...")
    try:
        from utils.tests.test_prompt_utils import TestPromptUtils
        import unittest
        
        # Create test suite and run extract_python_code tests
        suite = unittest.TestLoader().loadTestsFromTestCase(TestPromptUtils)
        # Suppress test output by redirecting to null device
        import os
        result = unittest.TextTestRunner(stream=open(os.devnull, 'w')).run(suite)
        
        if result.wasSuccessful():
            print("  ✅ extract_python_code tests - PASSED")
            test_results['extract_python_code'] = True
        else:
            print("  ❌ extract_python_code tests - FAILED")
            print(f"     Failures: {len(result.failures)}, Errors: {len(result.errors)}")
            test_results['extract_python_code'] = False
            
    except Exception as e:
        print(f"  ❌ extract_python_code tests - ERROR: {e}")
        test_results['extract_python_code'] = False
    
    # Test calculate_task_metrics using existing tests
    print("\n📈 Testing calculate_task_metrics (from utils/tests)...")
    try:
        from utils.tests.test_metrics_voting_integration import TestMetricsVotingIntegration
        
        # Run a specific test to validate metrics calculation
        test_instance = TestMetricsVotingIntegration()
        test_instance.test_single_test_case_voting_success()
        test_instance.test_edge_cases()
        
        print("  ✅ calculate_task_metrics tests - PASSED")
        test_results['calculate_task_metrics'] = True
        
    except Exception as e:
        print(f"  ❌ calculate_task_metrics tests - ERROR: {e}")
        test_results['calculate_task_metrics'] = False
    
    # Test basic grid scoring logic (simple inline test since no dedicated test exists)
    print("\n📊 Testing GridScorer (basic validation)...")
    try:
        scorer = GridScorer()
        
        # Test perfect match
        result1 = scorer.score_grid([[1, 2], [3, 4]], [[1, 2], [3, 4]])
        assert result1["correct"] == True, "Perfect match should be correct"
        
        # Test mismatch
        result2 = scorer.score_grid([[1, 2], [3, 4]], [[1, 2], [3, 5]])
        assert result2["correct"] == False, "Mismatch should be incorrect"
        
        print("  ✅ GridScorer basic tests - PASSED")
        test_results['grid_scorer'] = True
        
    except Exception as e:
        print(f"  ❌ GridScorer basic tests - ERROR: {e}")
        test_results['grid_scorer'] = False
    
    # Test basic program execution (simple inline test)
    print("\n⚙️ Testing ProgramExecutor (basic validation)...")
    try:
        executor = ProgramExecutor(timeout=1.0, executor_type="unrestricted")
        
        # Test working code
        working_code = "def transform(grid): return [[c+1 for c in r] for r in grid]"
        result, error, timed_out = executor.execute_program_with_timeout(working_code, [[1, 2]])
        assert result == [[2, 3]], f"Expected [[2, 3]], got {result}"
        
        # Test broken code
        broken_code = "def transform(grid): return grid[999]"  # Index error
        result, error, timed_out = executor.execute_program_with_timeout(broken_code, [[1, 2]])
        assert result is None, "Broken code should return None"
        
        print("  ✅ ProgramExecutor basic tests - PASSED")
        test_results['program_executor'] = True
        
    except Exception as e:
        print(f"  ❌ ProgramExecutor basic tests - ERROR: {e}")
        test_results['program_executor'] = False
    
    return test_results

# Run the existing tests
existing_test_results = run_existing_tests()


🧪 TESTING EVALUATION METRICS AND FUNCTIONS

📋 Running existing tests from utils/tests...

🐍 Testing extract_python_code (from utils/tests)...
  ✅ extract_python_code tests - PASSED

📈 Testing calculate_task_metrics (from utils/tests)...
  ✅ calculate_task_metrics tests - PASSED

📊 Testing GridScorer (basic validation)...
  ✅ GridScorer basic tests - PASSED

⚙️ Testing ProgramExecutor (basic validation)...
  ✅ ProgramExecutor basic tests - PASSED


In [39]:
def test_compute_arc_metrics_with_real_data():
    """Test the multi-attempt compute_arc_metrics function using real trainer and data"""
    print("\n🎯 Testing MULTI-ATTEMPT compute_arc_metrics with REAL trainer data...")
    
    # DEBUG: Check what's actually in the eval dataset
    print("🔍 DEBUG: Checking eval dataset structure...")
    sample = trainer.eval_dataset[0]
    print(f"  Available keys: {list(sample.keys())}")
    
    # Convert dataset to list to avoid indexing issues
    eval_data_list = [trainer.eval_dataset[i] for i in range(len(trainer.eval_dataset))]
    print(f"  Converted {len(eval_data_list)} examples to list")
    
    # Use REAL validation data from the trainer (all tasks)
    real_eval_data = eval_data_list
    
    # Temporarily replace eval_dataset for testing
    original_eval_dataset = trainer.eval_dataset
    trainer.eval_dataset = real_eval_data
    
    try:
        print(f"  Actually generating {NUM_EVAL_ATTEMPTS} attempts per task using the model...")
        
        # Extract input_ids from the converted list
        input_ids_list = []
        for sample in eval_data_list:
            # Convert list to tensor if needed
            if isinstance(sample['input_ids'], list):
                input_ids = torch.tensor(sample['input_ids'])
            else:
                input_ids = sample['input_ids']
            input_ids_list.append(input_ids)
        
        # Stack and pad if necessary
        max_len = max(len(ids) for ids in input_ids_list)
        padded_input_ids = []
        for ids in input_ids_list:
            if len(ids) < max_len:
                # Pad with tokenizer pad token
                padding = torch.full((max_len - len(ids),), tokenizer.pad_token_id)
                ids = torch.cat([ids, padding])
            padded_input_ids.append(ids)
        
        # Create inputs tensor
        inputs = {
            'input_ids': torch.stack(padded_input_ids).to(trainer.model.device)
        }
        
        print(f"  Input shape: {inputs['input_ids'].shape}")
        
        # Actually call VllmSFTTrainer.prediction_step to get REAL generations
        print("  Calling VllmSFTTrainer.prediction_step...")
        _, real_predictions, _ = trainer.prediction_step(
            model=trainer.model,
            inputs=inputs,
            prediction_loss_only=False
        )
        
        print(f"  Generated predictions shape: {real_predictions.shape}")
        
        # Create EvalPrediction object with REAL predictions
        eval_pred = EvalPrediction(
            predictions=real_predictions,
            label_ids=None
        )
        
        # Test the compute_arc_metrics function with real predictions
        metrics = compute_arc_metrics(eval_pred, debug=True)
        
        # Rest of validation code...
        
        # Verify expected structure for multi-attempt metrics
        expected_keys = [
            "all_test_correct", "all_train_correct", "min1_train_correct",
            "min1_code_success", "weighted_voting_pass2", "train_majority_pass2",
            "max_length_rate", "timeout_rate", "total_tasks", "total_responses", "total_attempts"
        ]
        
        passed = True
        for key in expected_keys:
            if key not in metrics:
                print(f"  ❌ Missing metric: {key}")
                passed = False
            elif not isinstance(metrics[key], (int, float)):
                print(f"  ❌ Metric {key} has wrong type: {type(metrics[key])}")
                passed = False
        
        # Check that we have reasonable values for multi-attempt setup
        if metrics["total_tasks"] != len(trainer.eval_dataset):
            print(f"  ❌ Expected 8 tasks, got {metrics['total_tasks']}")
            passed = False
        
        expected_total_attempts = len(trainer.eval_dataset) * NUM_EVAL_ATTEMPTS  # len(trainer.eval_dataset) tasks × attempts each
        if metrics["total_attempts"] != expected_total_attempts:
            print(f"  ❌ Expected {expected_total_attempts} total attempts, got {metrics['total_attempts']}")
            passed = False
        
        # Check that voting metrics are present (even if 0)
        if "weighted_voting_pass2" not in metrics:
            print(f"  ❌ Missing weighted_voting_pass2 metric (multi-attempt feature)")
            passed = False
        
        if "train_majority_pass2" not in metrics:
            print(f"  ❌ Missing train_majority_pass2 metric (multi-attempt feature)")
            passed = False
        
        # Verify sampling parameters are working (basic check)
        if metrics["min1_code_success"] < 0 or metrics["min1_code_success"] > 1:
            print(f"  ❌ Invalid min1_code_success: {metrics['min1_code_success']} (should be 0-1)")
            passed = False
        
        if passed:
            print("  ✅ compute_arc_metrics with real data - PASSED")
            print(f"     Multi-attempt metrics:")
            print(f"       Oracle (best): {metrics['all_test_correct']:.3f}")
            print(f"       Code success: {metrics['min1_code_success']:.3f}")
            print(f"       Voting pass@2: {metrics['weighted_voting_pass2']:.3f}")
            print(f"       Total attempts: {metrics['total_attempts']} ({NUM_EVAL_ATTEMPTS} per task)")
        else:
            print("  ❌ compute_arc_metrics with real data - FAILED")
        
    except Exception as e:
        print(f"  ❌ compute_arc_metrics real data test - ERROR: {e}")
        import traceback
        traceback.print_exc()
        passed = False
    
    finally:
        # Restore original settings
        trainer.eval_dataset = original_eval_dataset
    
    return passed

def test_trainer_integration():
    """Test that trainer is properly set up and integrated"""
    print("\n🔧 Testing trainer integration...")
    
    passed = True
    
    # Test trainer setup
    try:
        assert hasattr(trainer, 'model'), "Trainer missing model"
        assert hasattr(trainer, 'tokenizer'), "Trainer missing tokenizer"  
        assert hasattr(trainer, 'eval_dataset'), "Trainer missing eval_dataset"
        assert hasattr(trainer, 'compute_metrics'), "Trainer missing compute_metrics"
        
        # Test eval dataset structure
        eval_sample = trainer.eval_dataset[0]
        required_fields = ['train_input', 'train_output', 'test_input', 'test_output']
        for field in required_fields:
            if field not in eval_sample:
                print(f"  ❌ Missing field in eval dataset: {field}")
                passed = False
        
        # Test that compute_metrics is our function
        if trainer.compute_metrics != compute_arc_metrics:
            print(f"  ❌ Wrong compute_metrics function")
            passed = False
            
        # Test tokenizer compatibility
        if trainer.tokenizer != tokenizer:
            print(f"  ❌ Trainer tokenizer mismatch")
            passed = False
            
        print(f"  ✅ Trainer integration - PASSED")
        print(f"     Eval dataset size: {len(trainer.eval_dataset)}")
        print(f"     Tokenizer vocab size: {len(tokenizer.get_vocab())}")
        
    except Exception as e:
        print(f"  ❌ Trainer integration - ERROR: {e}")
        passed = False
    
    return passed

# Run trainer-specific tests
print("\n" + "="*60)
trainer_integration_passed = test_trainer_integration()
compute_metrics_real_passed = test_compute_arc_metrics_with_real_data()

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.




🔧 Testing trainer integration...
  ✅ Trainer integration - PASSED
     Eval dataset size: 32
     Tokenizer vocab size: 151669

🎯 Testing MULTI-ATTEMPT compute_arc_metrics with REAL trainer data...
🔍 DEBUG: Checking eval dataset structure...


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


  Available keys: ['reasoning', 'code', 'correct_train_input', 'train_input', 'train_output', 'predicted_train_output', 'correct_test_input', 'test_input', 'test_output', 'predicted_test_output', 'task_id', 'model', 'generation', 'messages', 'text', 'prompt', 'input_ids', 'attention_mask']
  Converted 32 examples to list
  Actually generating 8 attempts per task using the model...
  Input shape: torch.Size([32, 15051])
  Calling VllmSFTTrainer.prediction_step...
[vLLM predict] batch size = 32, generating 8 attempts per task


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

...

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/32 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


  Generated predictions shape: torch.Size([256, 1000])
🎯 MULTI-ATTEMPT EVALUATION: 8 attempts per task from trainer prediction_step
📊 Processing 256 predictions for 32 tasks (8 attempts each)

=== ARC task 0 - processing 8 pre-generated attempts ===
    Attempt 0: Generated text preview: 

```python
def transform(input_grid):
    # Determine the scaling factor based on the input grid size
    # For 2x2 input, the output is 6x6, so the scaling factor is 3
    scale = 3
    rows, cols =...
    Extracted code: def transform(input_grid):
    # Determine the scaling factor based on the input grid size
    # For...
    Program extracted: True
    Attempt 1: Generated text preview: 

```python
def transform(grid):
    # Transformation rule: Replace 8 with 2, 0 with 0, 1 with 0, 3 with 3, 7 with 7, 2 with 2, 4 with 4, 5 with 5, 6 with 6, 9 with 9
    # The transformation seems to...
    Extracted code: def transform(grid):
    # Transformation rule: Replace 8 with 2, 0 with 0, 1 with 0, 3 with

In [40]:
# Final evaluation of all test results
print("\n" + "="*60)
print("🏁 FINAL EVALUATION SYSTEM TEST SUMMARY")
print("="*60)

# Combine results from existing tests and trainer-specific tests
all_tests = [
    ("extract_python_code", existing_test_results.get('extract_python_code', False)),
    ("calculate_task_metrics", existing_test_results.get('calculate_task_metrics', False)),
    ("GridScorer", existing_test_results.get('grid_scorer', False)),
    ("ProgramExecutor", existing_test_results.get('program_executor', False)),
    ("trainer_integration", trainer_integration_passed),
    ("compute_arc_metrics (multi-attempt)", compute_metrics_real_passed),
]

passed_count = sum(1 for _, passed in all_tests if passed)
total_count = len(all_tests)

print(f"\n📊 TEST RESULTS:")
for test_name, passed in all_tests:
    status = "✅ PASSED" if passed else "❌ FAILED"
    print(f"  {test_name:<30} {status}")

print(f"\n🎯 OVERALL RESULTS:")
print(f"  Tests passed: {passed_count}/{total_count} ({passed_count/total_count:.1%})")

if passed_count == total_count:
    print("  🎉 ALL EVALUATION FUNCTIONS ARE WORKING CORRECTLY!")
    print("  ✅ Multi-attempt evaluation system ready for training!")
    print(f"  🔢 Training will use {NUM_EVAL_ATTEMPTS} attempts per task with temp=1.0, min_p=0.05")
elif passed_count >= total_count * 0.8:
    print("  ✅ Most evaluation functions are working correctly.")
    print("  ⚠️  Review the failed tests before proceeding with training.")
else:
    print("  ❌ Multiple evaluation functions have issues.")
    print("  🚨 STRONGLY RECOMMEND fixing these before training!")

print(f"\n🔧 WHAT THESE TESTS VALIDATED:")
print("  • Grid comparison logic (exact matching)")
print("  • Code execution with timeout handling")
print("  • Python code extraction from model outputs")
print("  • Multi-attempt generation with proper sampling (temp=1.0, min_p=0.05)")
print("  • Voting and best-attempt metrics across multiple attempts")
print("  • Training evaluation pipeline integration (compatible with run_arc_tasks_soar.py)")
print("  • Core utility functions (leveraging existing test suite)")

print(f"\n📈 MULTI-ATTEMPT EVALUATION METRICS:")
print(f"  • all_test_correct: Oracle (best attempt) - whether best attempt solves all test grids")
print(f"  • all_train_correct: Perfect training performance across attempts") 
print(f"  • min1_train_correct: Tasks with at least some training success")
print(f"  • min1_code_success: Tasks where at least one attempt generated working code")
print(f"  • weighted_voting_pass2: Pass@2 using weighted voting across attempts")
print(f"  • train_majority_pass2: Pass@2 using training majority voting") 
print(f"  • timeout_rate: Fraction of attempts that timed out")
print(f"  • max_length_rate: Fraction of attempts that hit max length")

print(f"\n🚀 MULTI-ATTEMPT TRAINING BENEFITS:")
print(f"  • Each validation task generates {NUM_EVAL_ATTEMPTS} attempts during evaluation")
print(f"  • Uses same sampling parameters as run_arc_tasks_soar.py (temp=1.0, min_p=0.05)")
print(f"  • Comprehensive metrics: oracle, voting, code success rates")
print(f"  • Better signal during training - can track improvement across multiple attempts")
print(f"  • Direct compatibility with existing ARC evaluation pipelines")

if passed_count == total_count:
    print(f"\n🚀 READY TO PROCEED WITH TRAINING!")
    print(f"   ✅ Multi-attempt evaluation system validated")
    print(f"   📊 Will generate {NUM_EVAL_ATTEMPTS} attempts per validation task")
    print(f"   🎯 Metrics include voting, oracle (best), and code success rates")
    print(f"   🔄 Sampling: temperature=1.0, min_p=0.05 (aligned with run_arc_tasks_soar.py)")
else:
    failed_tests = [name for name, passed in all_tests if not passed]
    print(f"\n⚠️  ISSUES TO INVESTIGATE:")
    for test_name in failed_tests:
        print(f"   • {test_name}")
    print(f"\n💡 RECOMMENDATION: Fix the failed tests before training to ensure accurate evaluation.")
    print(f"   💡 TIP: Run utils/tests/ directly for more detailed debugging of core functions.")

print("="*60)



🏁 FINAL EVALUATION SYSTEM TEST SUMMARY

📊 TEST RESULTS:
  extract_python_code            ✅ PASSED
  calculate_task_metrics         ✅ PASSED
  GridScorer                     ✅ PASSED
  ProgramExecutor                ✅ PASSED
  trainer_integration            ✅ PASSED
  compute_arc_metrics (multi-attempt) ✅ PASSED

🎯 OVERALL RESULTS:
  Tests passed: 6/6 (100.0%)
  🎉 ALL EVALUATION FUNCTIONS ARE WORKING CORRECTLY!
  ✅ Multi-attempt evaluation system ready for training!
  🔢 Training will use 8 attempts per task with temp=1.0, min_p=0.05

🔧 WHAT THESE TESTS VALIDATED:
  • Grid comparison logic (exact matching)
  • Code execution with timeout handling
  • Python code extraction from model outputs
  • Multi-attempt generation with proper sampling (temp=1.0, min_p=0.05)
  • Voting and best-attempt metrics across multiple attempts
  • Training evaluation pipeline integration (compatible with run_arc_tasks_soar.py)
  • Core utility functions (leveraging existing test suite)

📈 MULTI-ATTEMPT EVAL

In [41]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA H200. Max memory = 139.719 GB.
43.455 GB of memory reserved.


In [42]:
# We should consider training on completions only!!! which means the response part for the xentropy.

In [43]:
from unsloth.chat_templates import train_on_responses_only # or run the code above if not using unsloth

# TO SUPPORT REASONING, WE NEED TO DYNAMICALLY APPLY THE RIGHT MASKING, NOT YET IMPLEMENTED
# masks everything between the instruction_part and response_part
trainer = train_on_responses_only(
    trainer,
    instruction_part = instruction_tag,
    response_part = response_tag,
    # force_match=False # comment out to set true for a cleaner masking
)

Map (num_proc=192):   0%|          | 0/1600 [00:00<?, ? examples/s]

num_proc must be <= 32. Reducing num_proc to 32 for dataset of size 32.


Map (num_proc=32):   0%|          | 0/32 [00:00<?, ? examples/s]

In [44]:
tokenizer.decode(trainer.train_dataset[0]["input_ids"])

"<|im_start|>system\nYou are an AI assistant specialized in solving Abstract Reasoning Corpus (ARC-AGI) tasks by reasoning and generating Python code.<|im_end|>\n<|im_start|>user\nYou are an AI assistant specialized in solving Abstract Reasoning Corpus (ARC-AGI) tasks by generating Python code.\nYour goal is to analyze input-output grid pairs. The outputs were produced by applying a transformation rule to the inputs. Implement the transformation rules as a Python function.\nYou should only write the implemented the transformation in code.\nYou must write code in triple backticks (```python and then ```). You must write a function called 'transform' which takes a single argument, the input grid as 'list[list[int]]', and returns the transformed grid (also as 'list[list[int]]').\nYou should make sure that you implement a version of the transformation which works in general (at least for all given input-output pairs and test input pairs).\nThe number in the input grid can be mapped to the 

In [45]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[0]["labels"]]).replace(tokenizer.pad_token, " ")

'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,600 | Num Epochs = 3 | Total steps = 150
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 8 x 1) = 32
 "-____-"     Trainable parameters = 264,241,152 of 4,286,709,248 (6.16% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss,All Test Correct,All Train Correct,Min1 Train Correct,Min1 Code Success,Weighted Voting Pass2,Train Majority Pass2,Max Length Rate,Timeout Rate,Total Tasks,Total Responses,Total Attempts
15,0.416600,No log,0.000000,0.000000,0.031250,1.000000,0.000000,0.000000,0.000000,0.000000,32,256,256
30,0.240500,No log,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,32,256,256
45,0.218900,No log,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,32,256,256
60,0.148500,No log,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,32,256,256
75,0.132600,No log,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,32,256,256
90,0.122900,No log,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,32,256,256
105,0.106300,No log,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,32,256,256


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

[vLLM predict] batch size = 32, generating 8 attempts per task


...

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/32 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


🎯 MULTI-ATTEMPT EVALUATION: 8 attempts per task from trainer prediction_step
📊 Processing 256 predictions for 32 tasks (8 attempts each)
    Attempt 0: Generated text preview: 

```python
def transform(input_grid):
    # Determine the size of the output grid
    original_size = len(input_grid)
    output_size = original_size * 3  # Each original cell is repeated 3 times in ...
    Extracted code: def transform(input_grid):
    # Determine the size of the output grid
    original_size = len(input...
    Program extracted: True
    Attempt 1: Generated text preview: 

```python
def transform(grid):
    # Define the color mapping based on the given color codes
    color_map = {
        0: 0,  # Black
        1: 2,  # Blue -> 2
        2: 2,  # Red -> 2
        3: ...
    Extracted code: def transform(grid):
    # Define the color mapping based on the given color codes
    color_map = {...
    Program extracted: True
    Attempt 2: Generated text preview: 

```python
def transform(grid):
 

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
print(trainer_stats)

In [ ]:
# Check a few examples
for i in range(2):
    example = data["train"][i]
    print(f"\nExample {i} text length: {len(example['text'])}")
    print(f"Last 200 chars: {example['text'][-200:]}")

<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Qwen-3` team, the recommended settings for reasoning inference are `temperature = 0.6, top_p = 0.95, top_k = 20`

For normal chat based inference, `temperature = 0.7, top_p = 0.8, top_k = 20`

In [ ]:
# Legacy inference testing example - replaced by actual test in cell 61

In [ ]:
# print(tokenizer.chat_template)

In [ ]:
data["validation"]['prompt'][0]

In [ ]:
messages = [
    {"role" : "system", "content" : "You are an expert at solving abstract reasoning puzzles. Write clean, efficient Python code."},
    {"role" : "user", "content" : "You are solving an ARC (Abstraction and Reasoning Corpus) task. \nI will show you training examples with input and output grids, plus a test input grid. Your task is to:\n\n1. **Analyze the training examples** to discover patterns that map input grids to output grids\n2. **Write a Python program** that implements your best understanding of the transformation  \n3. **DO NOT predict or generate the test output** - your job is only to write the transformation program\n4. **Attempt a solution** - even if the pattern isn't completely clear, provide your best hypothesis\n5. **Do not repeat the same transformation** - if you have already tried a transformation, do not repeat it.\n\n**IMPORTANT: Your transformation must always produce a 10\u00d710 output grid.**\n\nThe test input is shown for context so you understand what type of grid your program will eventually process. Focus on learning patterns from training examples and writing code that captures your understanding.\n\nTraining Examples:\n\nExample 1:\nInput:\n5 0 0 5 0 0 0 5 0 0\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\nOutput:\n5 0 0 5 0 0 0 5 0 0\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n2 0 0 2 0 0 0 2 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n2 0 0 2 0 0 0 2 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n\nExample 2:\nInput:\n0 5 0 5 5 0 0 5 0 0\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\nOutput:\n0 5 0 5 5 0 0 5 0 0\n0 0 0 0 0 0 0 0 0 0\n0 2 0 2 2 0 0 2 0 5\n0 0 0 0 0 0 0 0 0 0\n0 2 0 2 2 0 0 2 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n0 2 0 2 2 0 0 2 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n\nExample 3:\nInput:\n0 0 5 5 0 5 0 5 5 0\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 0\nOutput:\n0 0 5 5 0 5 0 5 5 0\n0 0 0 0 0 0 0 0 0 0\n0 0 2 2 0 2 0 2 2 5\n0 0 2 2 0 2 0 2 2 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0\n0 0 2 2 0 2 0 2 2 5\n0 0 0 0 0 0 0 0 0 0\n0 0 2 2 0 2 0 2 2 5\n0 0 0 0 0 0 0 0 0 0\n\nTest Input:\n5 0 5 5 0 0 5 0 5 0\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 5\n\nAnalyze the patterns in the training examples and write a Python function that performs this transformation.\n\n**Approach Guidelines:**\n- Look for patterns in shapes, colors, positions, sizes, rotations, reflections, etc.\n- Even if you can't solve all training examples perfectly, implement what patterns you do observe\n- A partial solution that captures some aspects is better than returning the input unchanged\n- If the pattern is unclear, make your best educated guess based on what you can see\n\nRequirements:\n- The function takes a 2D list (grid) where grid[row][col] gives the value at that position\n- Values are integers from 0-9\n- Return a new grid (2D list) with the transformation applied\n- You can use numpy if needed - just add 'import numpy as np' at the start of your function\n- Aim to handle the training examples as well as possible, even if not perfectly\n- Your function should attempt some meaningful transformation based on the patterns you observe\n\nYou MUST end your response with the following exact format:\n\nFinal answer:\n```python\ndef transform(grid):\n    # Your transformation logic here (implement your best understanding)\n    return transformed_grid\n```\n"}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Disable thinking
)

# from transformers import TextStreamer
# _ = model.generate(
#     **tokenizer(text, return_tensors = "pt").to("cuda"),
#     max_new_tokens = 8000, # Increase for longer outputs!
#     # temperature = 0.6, top_p = 0.95, top_k = 20, # For thinking
#     temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
#     # temperature = 0.01,
#     streamer = TextStreamer(tokenizer, skip_prompt = True),
# )

# text = data["validation"]['prompt'][0]

from transformers import TextStreamer

inputs = tokenizer(text, return_tensors="pt").to("cuda")
input_ids = inputs["input_ids"]  # Extract for convenience

output_ids = model.generate(
    **inputs,
    max_new_tokens=2000,
    # temperature = 0.6, top_p = 0.95, top_k = 20, # For thinking
    # temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    temperature=0.1, # BEST FOR SINGLE ATTEMPTS
)

# Slice to skip the prompt portion in output
generated_tokens = output_ids[0][input_ids.shape[-1]:]
generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print(generated_text)

In [ ]:
# Use extract_python_code from utils (SOAR approach)
code = extract_python_code(generated_text)

if code:
    print(code)
    exec(code, globals())  # Defines `transform()` in global scope
else:
    raise ValueError("Could not extract Python code from generated text")

In [ ]:
# Alternative transform implementations commented out - using model generated version above


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# -------------------- helper --------------------
def safe_transform(grid):
    grid = grid.copy()        # <‑‑ clone so the original stays unchanged
    try:
        return transform(grid)
    except Exception as err:
        print(f"[safe_transform] transform() failed – {err}")
        return np.zeros_like(grid)

# -------------------- test case -----------------
test_case = {
    "input": np.array([  # convert to np.array for convenience
        [5, 0, 5, 5, 0, 0, 5, 0, 5, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 5],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 5],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 5],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 5],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 5]
    ]),
    "output": np.array([
        [5, 0, 5, 5, 0, 0, 5, 0, 5, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [2, 0, 2, 2, 0, 0, 2, 0, 2, 5],
        [2, 0, 2, 2, 0, 0, 2, 0, 2, 5],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [2, 0, 2, 2, 0, 0, 2, 0, 2, 5],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [2, 0, 2, 2, 0, 0, 2, 0, 2, 5],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [2, 0, 2, 2, 0, 0, 2, 0, 2, 5]
    ])
}

# -------------------- run & plot ----------------
predicted_output = safe_transform(test_case["input"])

fig, axs = plt.subplots(1, 3, figsize=(12, 4))
titles = ["Input", "Predicted Output", "Ground Truth Output"]
grids  = [test_case["input"], predicted_output, test_case["output"]]

for ax, grid, title in zip(axs, grids, titles):
    im = ax.imshow(grid, cmap="viridis", vmin=0, vmax=5)
    ax.set_title(title)
    ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Inference testing section ends here

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
# !pip show transformers

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

You can use this also to load a checkpoint!!! i.e. an intermediate checkpoint from training, so you can then push it to hub.

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        # max_seq_length = 30000,
        load_in_4bit = False,
    )

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
from huggingface_hub import HfFolder, login

# Call this at the top of your script / notebook
if HfFolder.get_token() is None:   # no token cached or in $HF_TOKEN
    login()                        # interactive prompt

In [ ]:
# print(model)

In [ ]:
# run_name = "Qwen2.5-Coder-7B-Instruct-gemini_synth_50_random_split_1_training-20250723-113848"
print(f"Pushing to Trelis/{run_name}")

In [ ]:
model = model.merge_and_unload()
model.push_to_hub(f"Trelis/{run_name}")
tokenizer.push_to_hub(f"Trelis/{run_name}")

In [ ]:
# # Merge to 16bit
# if False:
#     model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
# if True: # Pushing to HF Hub
#     model.push_to_hub_merged(f"Trelis/{run_name}", tokenizer, save_method = "merged_16bit")

# # Merge to 4bit
# if False:
#     model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
# if False: # Pushing to HF Hub
#     model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# # Just LoRA adapters
# if False:
#     model.save_pretrained("model")
#     tokenizer.save_pretrained("model")
# if False: # Pushing to HF Hub
#     model.push_to_hub("hf/model", token = "")
#     tokenizer.push_to_hub("hf/model", token = "")


In [ ]:
# print(tokenizer.chat_template)

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False:
    model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False:
    model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
